In [1]:
include("nn_ops.jl")

maxpoolconstraint

In [4]:
batch = 1
in_height = 5
in_width = 5
stride_height = 2
stride_width = 2
pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)
in_channels = 1
filter_height = 2
filter_width = 2
out_channels = 4

srand(2)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_conv_actual = conv2d(x_actual, filter)
x_conv_relu_actual = relu(x_conv_actual)
x_conv_relu_maxpool_actual = maxpool(x_conv_relu_actual, (1, 2, 2, 1));

In [8]:
m = Model(solver=GurobiSolver())

@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx_conv[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@variable(m, vx_conv_relu[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@variable(m, vx_conv_relu_maxpool[1:batch, 1:pooled_height, 1:pooled_width, 1:out_channels])
@constraint(m, vx .== x_current) # input

# 1. Only using convolution constraint
@constraint(m, conv2d(vx+ve, filter) .== vx_conv)
# @constraint(m, vx_conv .== x_conv_actual)

# 2. Adding relu layer
reluconstraint2(m, vx_conv, vx_conv_relu, 10000)
# @constraint(m, vx_conv_relu .== x_conv_relu_actual)

# 3. Adding maxpool layer
maxpoolconstraint(m, vx_conv_relu, vx_conv_relu_maxpool, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)


@objective(m, Min, sum(ve.^2))

print(m)

status = solve(m)

println("Objective value: ", getobjectivevalue(m))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(ve))

# TODO: fix the error message WARNING: A variable named a is already attached to this model. If creating variables programmatically, consider using the anonymous variable syntax x = @variable(m, [1:N], ...).

Min ve[1,1,1,1]² + ve[1,1,2,1]² + ve[1,1,3,1]² + ve[1,1,4,1]² + ve[1,1,5,1]² + ve[1,2,1,1]² + ve[1,2,2,1]² + ve[1,2,3,1]² + ve[1,2,4,1]² + ve[1,2,5,1]² + ve[1,3,1,1]² + ve[1,3,2,1]² + ve[1,3,3,1]² + ve[1,3,4,1]² + ve[1,3,5,1]² + ve[1,4,1,1]² + ve[1,4,2,1]² + ve[1,4,3,1]² + ve[1,4,4,1]² + ve[1,4,5,1]² + ve[1,5,1,1]² + ve[1,5,2,1]² + ve[1,5,3,1]² + ve[1,5,4,1]² + ve[1,5,5,1]²
Subject to
 vx[1,1,1,1] = -4
 vx[1,2,1,1] = 8
 vx[1,3,1,1] = -5
 vx[1,4,1,1] = 5
 vx[1,5,1,1] = -8
 vx[1,1,2,1] = 2
 vx[1,2,2,1] = -4
 vx[1,3,2,1] = -2
 vx[1,4,2,1] = -2
 vx[1,5,2,1] = 1
 vx[1,1,3,1] = 10
 vx[1,2,3,1] = 4
 vx[1,3,3,1] = 5
 vx[1,4,3,1] = -3
 vx[1,5,3,1] = -1
 vx[1,1,4,1] = 8
 vx[1,2,4,1] = -3
 vx[1,3,4,1] = 9
 vx[1,4,4,1] = 10
 vx[1,5,4,1] = -10
 vx[1,1,5,1] = 4
 vx[1,2,5,1] = -5
 vx[1,3,5,1] = 4
 vx[1,4,5,1] = -6
 vx[1,5,5,1] = 7
 -8 vx[1,1,1,1] - 8 ve[1,1,1,1] - 10 vx[1,2,1,1] - 10 ve[1,2,1,1] - 9 vx[1,1,2,1] - 9 ve[1,1,2,1] + 3 vx[1,2,2,1] + 3 ve[1,2,2,1] - vx_conv[1,1,1,1] = 0
 -8 vx[1,2,1,1] - 8

In [4]:
x_conv_current = conv2d(x_current+getvalue(ve), filter)
x_conv_relu_current = relu(x_conv_current)
x_conv_relu_maxpool_current = maxpool(x_conv_relu_current, (1, 2, 2, 1));

In [5]:
x_conv_current - x_conv_actual

1×6×6×4 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 5, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 6, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 5, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 6, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 1, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4

In [ ]:
x_conv_relu_current - x_conv_relu_actual

In [ ]:
x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual